In [1]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb

data = pd.read_csv('all_data_fixed3.csv', index_col=False)
data

,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Для выхода из экономического кризиса вызванног...,ввело 8 часовой рабочий день в промышленности,ликвидировало помещичье землевладение,отменило выкупные платежи,ввело продовольственную разверстку,4
5,Творчество поэтов Гумилева Ахматовой Мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1
6,Самое многочисленное сословие Российской империи,мещане,казачество,крестьянство,разночинцы,3
7,Усиление позиций России в Китае вызывало в пер...,США,Японии,Великобритании,Германии,2
8,Какое из названных событий произошло раньше др...,восстание на броненосце Князь Потемкин Тавриче...,восстание на крейсере Очаков,стачка текстильщиков в Иваново Вознесенске,созыв первой Государственной думы,3
9,Отметьте о ком идет речь Дворянин политический...,Столыпин,Милюков,Родзянко,Львов,3


In [16]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score

def get_samples(data, embeddings, train_part=0.75, random_seed=4):
    predata = np.zeros((embeddings.shape[0] * 4, embeddings.shape[2] * 2))
    target = []
    np.random.seed(random_seed)

    for i in range(len(data)):
        for j in range(1,5):
            cur_targ = 1 if j == int(data.iloc[i,5]) else 0
            cur_vec = np.hstack((embeddings[i,0], embeddings[i,j]))
            predata[i*4+j-1] = cur_vec
            target.append(cur_targ)
            
    all_inds = np.arange(embeddings.shape[0])
    train_size = int(len(all_inds) * train_part)
    train_inds = np.random.choice(all_inds, size=train_size, replace=False)
    test_inds = np.array(list(set(all_inds) - set(train_inds)))
    
    X_train, X_test, y_train, y_test = [],[],[],[]

    for i in range(len(train_inds)):
        for j in range(4):
            X_train.append(predata[train_inds[i] * 4 + j])
            y_train.append(target[train_inds[i] * 4 + j])

    for i in range(len(test_inds)):
        for j in range(4):
            X_test.append(predata[test_inds[i] * 4 + j])
            y_test.append(target[test_inds[i] * 4 + j])
            
    return X_train, X_test, y_train, y_test


def approach_logreg(data, embeddings, train_part=0.75, random_seed=0.4):
    
    X_train, X_test, y_train, y_test = get_samples(data, embeddings, train_part)
    
    clf = LR()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict_proba(X_test)
    y_pred_new = []

    for i in range(len(y_pred) // 4):
        cur_probs, preds = [], [0,0,0,0]
        for j in range(4):
            cur_probs.append(y_pred[i*4 + j][1])
        preds[np.argmax(cur_probs)] = 1
        y_pred_new.extend(preds)
        
    y_pred_splitted, y_test_splitted = [], []

    for i in range(len(y_pred) // 4):
        y_pred_splitted.append(np.argmax(y_pred_new[i*4:i*4+4]))
        y_test_splitted.append(np.argmax(y_test[i*4:i*4+4]))
        
    accuracy = accuracy_score(y_test_splitted, y_pred_splitted)
    
    return accuracy

In [17]:
import pickle

with open('elmo_embed.pickle', 'rb') as f:
    elmo_embs = pickle.load(f)

elmo_embs = np.array(elmo_embs)
approach_logreg(data, elmo_embs)

0.30682670667666917

In [18]:
import pickle

with open('fasttext_embed.pickle', 'rb') as f:
    fasttext_embs = pickle.load(f)

fasttext_embs = np.array(fasttext_embs)
approach_logreg(data, fasttext_embs)

0.29857464366091524

In [14]:
import pickle

with open('rusvec_embed.pickle', 'rb') as f:
    rusvec_embs = pickle.load(f)

rusvec_embs = np.array(rusvec_embs)
approach_logreg(data, rusvec_embs)

0.291072768192048

In [15]:
import pickle

with open('bert_embed3.pickle', 'rb') as f:
    bert_embs = pickle.load(f)

bert_embs = np.array(bert_embs)
approach_logreg(data, bert_embs)

0.2970742685671418